In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from datetime import datetime

from keras import models, optimizers, layers
from keras import backend as K
from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding,TimeDistributed,Conv1D,Input, Flatten,Reshape ,MaxPooling2D, Dropout,Activation, LSTM
from keras.callbacks import ModelCheckpoint,LearningRateScheduler


from keras import callbacks
from keras import initializers
from keras.preprocessing.image import ImageDataGenerator
import time

import glob
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

Using TensorFlow backend.


In [2]:
base_path = "/home/forest/data/LSTM_capsule/train/"
speechs = os.listdir(base_path )
from skimage.transform import resize

sizeA = 16000
sizeB = 1
cat_nums = 10
train_image = np.zeros((1, sizeA, sizeB))
train_label = [0]
for i in range(10):
    subfile = os.listdir(base_path + speechs[i] +"/")
    train_audio_path = base_path + speechs[i] +"/"
    size = len(subfile)
    out_image = np.zeros((size, sizeA, sizeB))
    out_label = np.full((size), i)
    for index in range(1, len(subfile)):
        sample_rate, samples = wavfile.read(str(train_audio_path) + subfile[index])
        
        
        samples = np.float64(samples)
    
    
        log_S = samples/np.sqrt(np.sum(np.square((np.abs(samples))))/len(samples))
        where_are_NaNs = np.isnan(log_S)
        
        log_S[where_are_NaNs] = 0
        
        log_S = np.pad(log_S, (0, sizeA - len(log_S)), 'constant', constant_values=0)
        log_S = log_S.reshape((1, sizeA, sizeB))
        out_image[index] = log_S
    train_image = np.concatenate((train_image, out_image))
    train_label = np.concatenate((train_label, out_label))
x_data = train_image[2:]
y_data = train_label[2:]

In [3]:
sample_len = 16000
overlap = 2000
steps_len = 4000
steps = 7

In [4]:
data = []
for i in range(steps):
    data.append(x_data[:,(i*overlap):(i*overlap+steps_len)])
data = np.asarray(data).astype('float32')

print ('data.shape',data.shape)

data.shape (7, 19326, 4000, 1)


In [5]:
data_tran = np.transpose(data, (1, 0, 2, 3))

In [7]:
npad = ((0, 0), (0, 0), (0, 100), (0, 0))
data_pad = np.pad(data_tran, pad_width=npad, mode='constant', constant_values=0)

In [9]:
np.random.seed(42)
shuffle_indices = np.random.permutation(np.arange(len(y_data)))
split_num = round(len(y_data) * 0.1)

x_valid = data_pad[shuffle_indices[: split_num]]
y_valid = y_data[shuffle_indices[: split_num]]

x_test = data_pad[shuffle_indices[split_num: split_num*2]]
y_test = y_data[shuffle_indices[split_num: split_num*2]]

x_train = data_pad[shuffle_indices[split_num*2 :]]
y_train = y_data[shuffle_indices[split_num*2 :]]

In [12]:
x = Input(shape=[4100, 1])

In [14]:
conv1 =Conv1D(filters=128, kernel_size=101,strides=10, padding='valid', activation='relu', name='conv1')(x)

In [15]:
conv1_batchNorm = layers.BatchNormalization()(conv1)
conv1_act = layers.Activation('relu')(conv1_batchNorm)
conv1_drop = Dropout(0.5)(conv1_act)

In [16]:
def squash(vectors, axis=-1):

    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors

def PrimaryCap(inputs, dim_capsule, n_channels, kernel_size, strides, padding):
  
    output = layers.Conv1D(filters=dim_capsule*n_channels, kernel_size=kernel_size, strides=strides, padding=padding,
                           name='primarycap_conv1d')(inputs)
    
    conv2_batchNorm = layers.BatchNormalization()(output)

    outputs = layers.Reshape(target_shape=[-1, dim_capsule], name='primarycap_reshape')(conv2_batchNorm)
    return layers.Lambda(squash, name='primarycap_squash')(outputs)

In [17]:
primarycaps = PrimaryCap(conv1_drop, dim_capsule=8, n_channels=16, kernel_size=101, strides=10, padding='valid')

In [18]:
class CapsuleLayer(layers.Layer):

    def __init__(self, num_capsule, dim_capsule, num_routing=3,
                 kernel_initializer='glorot_uniform',
                 **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.num_routing = num_routing
        self.kernel_initializer = initializers.get(kernel_initializer)

    def build(self, input_shape):
        assert len(input_shape) >= 3, "The input Tensor should have shape=[None, input_num_capsule, input_dim_capsule]"
        self.input_num_capsule = input_shape[1]
        self.input_dim_capsule = input_shape[2]

        # Transform matrix
        self.W = self.add_weight(shape=[self.num_capsule, self.input_num_capsule,
                                        self.dim_capsule, self.input_dim_capsule],
                                 initializer=self.kernel_initializer,
                                 name='W')

        self.built = True

    def call(self, inputs, training=None):

        inputs_expand = K.expand_dims(inputs, 1)

  
        inputs_tiled = K.tile(inputs_expand, [1, self.num_capsule, 1, 1])

  
        inputs_hat = K.map_fn(lambda x: K.batch_dot(x, self.W, [2, 3]), elems=inputs_tiled)


        inputs_hat_stopped = K.stop_gradient(inputs_hat)
        
        b = tf.zeros(shape=[K.shape(inputs_hat)[0], self.num_capsule, self.input_num_capsule])

        assert self.num_routing > 0, 'The num_routing should be > 0.'
        for i in range(self.num_routing):
            c = tf.nn.softmax(b, dim=1)

            if i == self.num_routing - 1:
   
                outputs = squash(K.batch_dot(c, inputs_hat, [2, 2]))  # [None, 10, 16]
            else:  
                outputs = squash(K.batch_dot(c, inputs_hat_stopped, [2, 2]))


        return outputs

    def compute_output_shape(self, input_shape):
        return tuple([None, self.num_capsule, self.dim_capsule])

In [19]:
digitcaps = CapsuleLayer(num_capsule=10, dim_capsule=16, num_routing=3,
                             name='digitcaps')(primarycaps)

Instructions for updating:
dim is deprecated, use axis instead


In [20]:
digitcaps

<tf.Tensor 'digitcaps/mul_2:0' shape=(?, 10, 16) dtype=float32>

In [21]:
out_caps = Flatten()(digitcaps)

In [22]:
capsnet = models.Model(x,  out_caps)

In [23]:
model = Sequential()
model.add(TimeDistributed(capsnet,input_shape = (7,4100,1)))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

In [24]:
model.compile(optimizer=optimizers.RMSprop(lr=0.001),loss='mse',metrics = ['accuracy'])

In [25]:
def one_hot(y_, n_classes=10):
    
    y_ = y_.reshape(len(y_))
    return np.eye(n_classes)[np.array(y_, dtype=np.int32)]  

In [26]:
x_train.shape

(15460, 7, 4100, 1)

In [ ]:
conv12_batchNorm_act_drop_history = model.fit(x_train, one_hot(y_train),
          batch_size =40,
          epochs=20, validation_data=(x_test, one_hot(y_test)))

In [ ]:
score = model.evaluate(x_test, one_hot(y_test))

In [ ]:
score